In [48]:
import requests
import json

def _parse_stock_string(stock_str):
    # 定义已知市场类型（可扩展）
    known_markets = {"港股", "美股", "股票", "A股"}
    parts = stock_str.split()
    if len(parts) < 3:
        return None  
    code_part = parts[0]
    if "||" in code_part:
        stock_code = code_part.split("||")[-1]
    else:
        stock_code = code_part
    stock_market = 'A股' if parts[-1] =='股票' else parts[-1]
    stock_name = parts[1]

    return {
        "stock_code": stock_code,
        "stock_name": stock_name,
        "stock_market": stock_market
    }

def _parse_stock_data(json_data):
    results = []
    for sublist in json_data:
        if not isinstance(sublist, list):
            continue
            
        for stock_str in sublist:
            if not isinstance(stock_str, str):
                continue
                
            stock_info = _parse_stock_string(stock_str)
            if stock_info:
                results.append(stock_info)
    
    return results[0] if len(results) > 0 else []

def parse_stock_input(input):
    url = f'https://news.10jqka.com.cn/public/index_keyboard_{input.strip()}_stock,hk,usa_5_jsonp.html'
    headers = {
        "referer": "https://stockpage.10jqka.com.cn/",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    jsonp_data = response.text
    if jsonp_data.startswith('jsonp(') and jsonp_data.endswith(')'):
        json_str = jsonp_data[6:-1]  
    else:
        json_str = jsonp_data
    parsed_data = json.loads(json_str)
    standard_json = json.dumps(parsed_data, ensure_ascii=False, indent=2)
    data = json.loads(standard_json)
    return _parse_stock_data(data)
parse_stock_input('HK2018')

{'stock_code': '02018', 'stock_name': '瑞声科技', 'stock_market': '港股'}